In [5]:
#for removing future warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os
import random
from tensorflow.python.platform import gfile
import librosa
import librosa.display as ld
from array import array
import soundfile
import sounddevice as sd

Import Keras seperti ini

In [6]:
# from tf.keras.models import Sequential  # Tidak bisa!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import Embedding, LSTM

In [7]:
print(tf.__version__)
print(tf.keras.__version__)

1.6.0
2.1.3-tf


# KONSTANTA & VARIABEL

In [8]:
DEBUG = False
PATH_TRAIN = os.getcwd()+'/data/train'
PATH_TEST = os.getcwd()+'/data/test'
LOGDIR = 'log/'
TEST_LOGDIR = 'log_test/'
LABEL_TO_INDEX_MAP = {}
WIDTH = 1025
HEIGHT = 1
NUM_LABELS = 0
BATCHRUN = 500
EPOCHS = 5
msframe = 320 #jumlah frame yang mau dianalisa
sequence = 5 #jumlah deret yang di ambil
DURATION = 20
gain=10
high=2000
low=100

In [11]:
def init(path):
    labels = os.listdir(path)
    index = 0
    for label in labels:
        LABEL_TO_INDEX_MAP[label] = index
        index += 1
        
    global NUM_LABELS
    NUM_LABELS = len(LABEL_TO_INDEX_MAP)
    print(LABEL_TO_INDEX_MAP)

def one_hot_encoding(label):
    encoding = [0] * len(LABEL_TO_INDEX_MAP)
    encoding[LABEL_TO_INDEX_MAP[label]] = 1
    return encoding

def get_batch(batch_size, path):
    X = []
    Y = []
    random.seed(5896)
    path = os.path.join(path,'*', '*.wav')
    waves = gfile.Glob(path)
    while True:
        random.shuffle(waves)
        for wave_path in waves:
            _,label = os.path.split(os.path.dirname(wave_path))
            X.append(get_spec(wave_path))
            Y.append(one_hot_encoding(label))

            if (len(X) == batch_size):
                yield X, Y
                X = []
                Y = []

def get_spec(wave_path, PAD_WIDTH=WIDTH):
    mg=[]
    wv, sr = soundfile.read(wave_path)
    delta_f = (high - low) / (80 - 1)
    fftsize = np.ceil(sr / delta_f).astype(int)
    low_bin = int(np.floor(low / delta_f))

    
    #print(wave_path)
    blocksize = int(sr * DURATION / 1000)
    #print('block:',blocksize)
    for i in range(0,len(wv),blocksize):
        magnitude = np.abs(np.fft.rfft(wv[i:i+blocksize], n=fftsize))
        magnitude *= gain / fftsize
        mg.append(magnitude)
    return np.array(mg)

### Untuk memeriksa

In [10]:
a = get_spec('mytest-16000.wav')

In [11]:
len(a[0])

334

In [10]:
import IPython.display as ipd
ipd.Audio('mytest-16000.wav')

# Keras Model

In [143]:
model = Sequential()
#model3.add(Embedding(20000,128))

model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=False))
model.add(Dense(6,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [144]:
from tensorflow.python.keras.optimizers import Adam
optimizer = Adam(lr=1e-4)

# Training

In [124]:
init(PATH_TRAIN)
batch = get_batch(BATCHRUN,PATH_TRAIN)
X, Y = next(batch)

x_train=[]
y_train=[]

for i in range(len(X)):
    ds=[]
    for ll in range(len(X[i])-sequence):
        ds = X[i][ll:ll+sequence]
        x_train.append(np.array(ds))
        y_train.append(Y[i])

#MODEL TRAINING
model.fit(x=np.array(x_train), y=np.array(y_train), dtype='float', epochs=20, verbose=2)
print('...Training done')

{'five': 0, 'four': 1, 'one': 2, 'six': 3, 'three': 4, 'two': 5}
Epoch 1/20
 - 35s - loss: 1.5994 - acc: 0.3147
Epoch 2/20
 - 31s - loss: 1.5140 - acc: 0.3636
Epoch 3/20
 - 31s - loss: 1.4721 - acc: 0.3830
Epoch 4/20
 - 31s - loss: 1.4356 - acc: 0.3955
Epoch 5/20
 - 31s - loss: 1.4052 - acc: 0.4115
Epoch 6/20
 - 31s - loss: 1.3851 - acc: 0.4222
Epoch 7/20
 - 31s - loss: 1.3632 - acc: 0.4389
Epoch 8/20
 - 31s - loss: 1.3440 - acc: 0.4403
Epoch 9/20
 - 31s - loss: 1.3206 - acc: 0.4547
Epoch 10/20
 - 31s - loss: 1.3057 - acc: 0.4651
Epoch 11/20
 - 31s - loss: 1.2866 - acc: 0.4706
Epoch 12/20
 - 31s - loss: 1.2714 - acc: 0.4810
Epoch 13/20
 - 31s - loss: 1.2517 - acc: 0.4905
Epoch 14/20
 - 31s - loss: 1.2332 - acc: 0.4952
Epoch 15/20
 - 31s - loss: 1.2174 - acc: 0.5028
Epoch 16/20
 - 31s - loss: 1.2073 - acc: 0.5061
Epoch 17/20
 - 31s - loss: 1.1882 - acc: 0.5153
Epoch 18/20
 - 31s - loss: 1.1742 - acc: 0.5220
Epoch 19/20
 - 31s - loss: 1.1575 - acc: 0.5280
Epoch 20/20
 - 31s - loss: 1.151

Epoch 1/20
 - 32s - loss: 0.5652 - acc: 0.7819
Epoch 2/20
 - 32s - loss: 0.5685 - acc: 0.7766
Epoch 3/20
 - 32s - loss: 0.5743 - acc: 0.7783
Epoch 4/20
 - 32s - loss: 0.5699 - acc: 0.7771
Epoch 5/20
 - 32s - loss: 0.5599 - acc: 0.7804
Epoch 6/20
 - 32s - loss: 0.5601 - acc: 0.7821
Epoch 7/20
 - 32s - loss: 0.5562 - acc: 0.7837
Epoch 8/20
 - 32s - loss: 0.5578 - acc: 0.7812
Epoch 9/20
 - 32s - loss: 0.5495 - acc: 0.7841
Epoch 10/20
 - 32s - loss: 0.5549 - acc: 0.7862
Epoch 11/20
 - 32s - loss: 0.5553 - acc: 0.7831
Epoch 12/20
 - 32s - loss: 0.5467 - acc: 0.7842
Epoch 13/20
 - 32s - loss: 0.5448 - acc: 0.7871
Epoch 14/20
 - 33s - loss: 0.5473 - acc: 0.7888
Epoch 15/20
 - 31s - loss: 0.5438 - acc: 0.7881
Epoch 16/20
 - 31s - loss: 0.5430 - acc: 0.7905
Epoch 17/20
 - 31s - loss: 0.5455 - acc: 0.7893
Epoch 18/20
 - 31s - loss: 0.5400 - acc: 0.7876
Epoch 19/20
 - 31s - loss: 0.5359 - acc: 0.7914
Epoch 20/20
 - 31s - loss: 0.5332 - acc: 0.7934
{'five': 0, 'four': 1, 'one': 2, 'six': 3, 'three

Epoch 3/20
 - 31s - loss: 0.4142 - acc: 0.8420
Epoch 4/20
 - 31s - loss: 0.4018 - acc: 0.8466
Epoch 5/20
 - 31s - loss: 0.4040 - acc: 0.8438
Epoch 6/20
 - 31s - loss: 0.4054 - acc: 0.8444
Epoch 7/20
 - 31s - loss: 0.4120 - acc: 0.8427
Epoch 8/20
 - 31s - loss: 0.4068 - acc: 0.8445
Epoch 9/20
 - 31s - loss: 0.4011 - acc: 0.8467
Epoch 10/20
 - 31s - loss: 0.4036 - acc: 0.8473
Epoch 11/20
 - 31s - loss: 0.4006 - acc: 0.8443
Epoch 12/20
 - 31s - loss: 0.4015 - acc: 0.8446
Epoch 13/20
 - 31s - loss: 0.4020 - acc: 0.8446
Epoch 14/20
 - 33s - loss: 0.3954 - acc: 0.8454
Epoch 15/20
 - 34s - loss: 0.4014 - acc: 0.8458
Epoch 16/20
 - 33s - loss: 0.4015 - acc: 0.8490
Epoch 17/20
 - 33s - loss: 0.3981 - acc: 0.8478
Epoch 18/20
 - 34s - loss: 0.3995 - acc: 0.8488
Epoch 19/20
 - 33s - loss: 0.3949 - acc: 0.8493
Epoch 20/20
 - 32s - loss: 0.3930 - acc: 0.8480
{'five': 0, 'four': 1, 'one': 2, 'six': 3, 'three': 4, 'two': 5}
Epoch 1/20
 - 36s - loss: 0.4022 - acc: 0.8469
Epoch 2/20
 - 37s - loss: 0.396

KeyboardInterrupt: 

# TEST PREDIKSI

In [1]:
magnitudo=[]
audiodata=[]

def pred(length=1, filename=None, thres=0):
    global cumulated_status, cseq, model,magnitudo, audiodata
    magnitudo=[]
    audiodata=[]

    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / (80 - 1)
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()
        cseq = 0
        def callback(indata, frames, time, status):
            global cumulated_status, cseq, magnitudo
            cumulated_status |= status
            
            cseq += 1
            
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize
                audiodata.extend(itertools.chain(indata.tolist()))

                magnitudo.append(magnitude)
                
                if cseq == 5:
                    cseq = 0
                    try:
                        y = model.predict(x=np.array(magnitudo).reshape(1,5,334))
                        cls_pred = np.argmax(y,axis=1)
                        #print(".")
                        if y.max()>0.94:
                            ##print('.')
                            print(list(LABEL_TO_INDEX_MAP.keys())[cls_pred[0]])
                    except Exception as e:
                        print(e)
                        pass
                    magnitudo=[]
                    
        t0=time.time()
        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            
            while True:
                t1=time.time()
                if t1-t0 > float(length):
                    if filename!=None: soundfile.write(filename,audiodata,16000)
                    return np.array(magnitudo)


        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)

In [21]:
import time
import itertools
a = pred(length=5)

.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' is not defined
.
name 'y' i

In [147]:
y_pred = model.predict(x=np.array(a).reshape(1,1,334))

In [181]:
list(LABEL_TO_INDEX_MAP.keys())[[1]])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [15]:
from tensorflow.python.keras.models import load_model
model = load_model('myfirst-model.ker')


NameError: name 'LABEL_TO_INDEX_MAP' is not defined